##embedding And vectorStoreDB

RAG Pipelines- Data Ingestion to Vector DB Pipeline

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

c:\Users\jasmi\OneDrive\Desktop\RAGtut\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 5 PDF files to process

Processing: 15 marks VDA.pdf
  ✓ Loaded 29 pages

Processing: bda ccet1.pdf
  ✓ Loaded 58 pages

Processing: Part B OE.pdf
  ✓ Loaded 19 pages

Processing: plm 15 marks.pdf
  ✓ Loaded 23 pages

Processing: UNIT-01_2M[1].pdf
  ✓ Loaded 9 pages

Total documents loaded: 138


In [3]:

all_pdf_documents

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-03-22T08:58:16+05:30', 'author': 'JASMINE DORATHY V', 'moddate': '2025-03-22T08:58:16+05:30', 'source': '..\\data\\pdf\\15 marks VDA.pdf', 'total_pages': 29, 'page': 0, 'page_label': '1', 'source_file': '15 marks VDA.pdf', 'file_type': 'pdf'}, page_content="15 marks VDA \n \n1.Significance of Software Standards in OpenGL and How It Ensures Cross-Platform Compatibility \nSignificance of Software Standards in OpenGL \nSoftware standards in OpenGL are essential for ensuring uniformity, performance, and compatibility \nacross different computing environments. OpenGL is an open standard for graphics rendering, which \nmeans it is maintained and updated by a consortium rather than a single company. The importance \nof software standards in OpenGL can be explained through the following points: \n1. Cross-Platform Consistency: \nOpenGL provides a standardized API that enables graphi

In [13]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [14]:
chunks=split_documents(all_pdf_documents)
chunks

Split 138 documents into 283 chunks

Example chunk:
Content: 15 marks VDA 
 
1.Significance of Software Standards in OpenGL and How It Ensures Cross-Platform Compatibility 
Significance of Software Standards in OpenGL 
Software standards in OpenGL are essential...
Metadata: {'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-03-22T08:58:16+05:30', 'author': 'JASMINE DORATHY V', 'moddate': '2025-03-22T08:58:16+05:30', 'source': '..\\data\\pdf\\15 marks VDA.pdf', 'total_pages': 29, 'page': 0, 'page_label': '1', 'source_file': '15 marks VDA.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-03-22T08:58:16+05:30', 'author': 'JASMINE DORATHY V', 'moddate': '2025-03-22T08:58:16+05:30', 'source': '..\\data\\pdf\\15 marks VDA.pdf', 'total_pages': 29, 'page': 0, 'page_label': '1', 'source_file': '15 marks VDA.pdf', 'file_type': 'pdf'}, page_content='15 marks VDA \n \n1.Significance of Software Standards in OpenGL and How It Ensures Cross-Platform Compatibility \nSignificance of Software Standards in OpenGL \nSoftware standards in OpenGL are essential for ensuring uniformity, performance, and compatibility \nacross different computing environments. OpenGL is an open standard for graphics rendering, which \nmeans it is maintained and updated by a consortium rather than a single company. The importance \nof software standards in OpenGL can be explained through the following points: \n1. Cross-Platform Consistency: \nOpenGL provides a standardized API that enables graphi


##embedding And vectorStoreDB

In [7]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\jasmi\OneDrive\Desktop\RAGtut\.venv\Lib\site-packages\chromadb\execution\expression\operator.py:239: SyntaxWarning: invalid escape sequence '\.'
  Key("email").regex(r".*@example\.com")


In [8]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2


c:\Users\jasmi\OneDrive\Desktop\RAGtut\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jasmi\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Fall

Model loaded successfully. Embedding dimension: 384



VectorStore

In [9]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [15]:
chunks

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-03-22T08:58:16+05:30', 'author': 'JASMINE DORATHY V', 'moddate': '2025-03-22T08:58:16+05:30', 'source': '..\\data\\pdf\\15 marks VDA.pdf', 'total_pages': 29, 'page': 0, 'page_label': '1', 'source_file': '15 marks VDA.pdf', 'file_type': 'pdf'}, page_content='15 marks VDA \n \n1.Significance of Software Standards in OpenGL and How It Ensures Cross-Platform Compatibility \nSignificance of Software Standards in OpenGL \nSoftware standards in OpenGL are essential for ensuring uniformity, performance, and compatibility \nacross different computing environments. OpenGL is an open standard for graphics rendering, which \nmeans it is maintained and updated by a consortium rather than a single company. The importance \nof software standards in OpenGL can be explained through the following points: \n1. Cross-Platform Consistency: \nOpenGL provides a standardized API that enables graphi

In [16]:

### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 283 texts...


Batches: 100%|██████████| 9/9 [00:10<00:00,  1.18s/it]


Generated embeddings with shape: (283, 384)
Adding 283 documents to vector store...
Successfully added 283 documents to vector store
Total documents in collection: 283



Retriever Pipeline From VectorStore

In [17]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [18]:

rag_retriever

In [19]:

rag_retriever.retrieve("What is attention is all you need")

Retrieving documents for query: 'What is attention is all you need'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 13.16it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)


[]